## gpu使用

从CPU到GPU

In [ ]:
device = torch.device("cuda")
tensor = tensor.to(device)
module.to(device)

从GPU到CPU

In [ ]:
device = torch.device(cpu)
tensor = tensor.to("cpu")
module.to("cpu")

tensor和module的 to()方法的区别是：tensor.to()执行的不是 inplace 操作，因此需要赋值；module.to()执行的是 inplace 操作。

转换数据类型

In [ ]:
x = torch.ones((3, 3))
x = x.to(torch.float64)

### 示例

In [4]:
import torch
import torch.nn as nn
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.cuda.is_available() else "cpu")

In [5]:
x_cpu = torch.ones((3, 3))
print("x_cpu:\ndevice: {} is_cuda: {} id: {}".format(
    x_cpu.device, x_cpu.is_cuda, id(x_cpu)))

x_gpu = x_cpu.to(device)
print("x_gpu:\ndevice: {} is_cuda: {} id: {}".format(
    x_gpu.device, x_gpu.is_cuda, id(x_gpu)))

x_cpu:
device: cpu is_cuda: False id: 5018624544
x_gpu:
device: cpu is_cuda: False id: 5018624544


In [6]:
net = nn.Sequential(nn.Linear(3, 3))

print("\nid:{} is_cuda: {}".format(id(net), next(net.parameters()).is_cuda))

net.to(device)
print("\nid:{} is_cuda: {}".format(id(net), next(net.parameters()).is_cuda))


id:4411039904 is_cuda: False

id:4411039904 is_cuda: False


- torch.cuda常用方法
    - torch.cuda.device_count()：返回当前可见可用的 GPU 数量
    - torch.cuda.get_device_name()：获取 GPU 名称
    - torch.cuda.manual_seed()：为当前 GPU 设置随机种子
    - torch.cuda.manual_seed_all()：为所有可见 GPU 设置随机种子
    - torch.cuda.set_device()：设置主 GPU 为哪一个物理 GPU，此方法不推荐使用
    - os.environ.setdefault("CUDA_VISIBLE_DEVICES", "2", "3")：设置可见 GPU
        - "2","3"表示的时物理GPU的编号
        - 有几个数字代表有几个逻辑GPU

### 多 GPU 的分发并行
- torch.nn.DataParallel(module, device_ids=None, output_device=None, dim=0)
- 功能：包装模型，实现分发并行机制。可以把数据平均分发到各个 GPU 上，每个 GPU 实际的数据量为 {batch_size}/{GPU 数量}，实现并行计算。
- 主要参数：
    - module：需要包装分发的模型
    - device_ids：可分发的 GPU，默认分发到所有可见可用的 GPU
    - output_device：结果输出设备
- 需要注意的是：使用 DataParallel 时，device 要指定某个 GPU 为 主 GPU，否则会报错：
    - RuntimeError: module must have its parameters and buffers on device cuda:1 (device_ids[0]) but found one of them on device: cuda:2
    - 这是因为，使用多 GPU 需要有一个主 GPU，来把每个 batch 的数据分发到每个 GPU，并从每个 GPU 收集计算好的结果。如果不指定主 GPU，那么数据就直接分发到每个 GPU，会造成有些数据在某个 GPU，而另一部分数据在其他 GPU，计算出错

设置两个可见 GPU，batch_size 为 2，那么每个 GPU 每个 batch 拿到的数据数量为 8，在模型的前向传播中打印数据的数量

In [ ]:
# 设置 2 个可见 GPU
gpu_list = [0,1]
gpu_list_str = ','.join(map(str, gpu_list))
os.environ.setdefault("CUDA_VISIBLE_DEVICES", gpu_list_str)
# 这里注意，需要指定一个 GPU 作为主 GPU。
# 否则会报错：module must have its parameters and buffers on device cuda:1 (device_ids[0]) but found one of them on device: cuda:2
# 参考：https://stackoverflow.com/questions/59249563/runtimeerror-module-must-have-its-parameters-and-buffers-on-device-cuda1-devi
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 16

# data
inputs = torch.randn(batch_size, 3)
labels = torch.randn(batch_size, 3)

inputs, labels = inputs.to(device), labels.to(device)

# model
net = FooNet(neural_num=3, layers=3)
net = nn.DataParallel(net)
net.to(device)

# training
for epoch in range(1):

    outputs = net(inputs)

    print("model outputs.size: {}".format(outputs.size()))

print("CUDA_VISIBLE_DEVICES :{}".format(os.environ["CUDA_VISIBLE_DEVICES"]))
print("device_count :{}".format(torch.cuda.device_count()))

"""
batch size in forward: 8
model outputs.size: torch.Size([16, 3])
CUDA_VISIBLE_DEVICES :0,1
device_count :2
"""

根据 GPU 剩余内存来排序

In [ ]:
def get_gpu_memory():
    import platform
    if 'Windows' != platform.system():
        import os
        os.system('nvidia-smi -q -d Memory | grep -A4 GPU | grep Free > tmp.txt')
        memory_gpu = [int(x.split()[2]) for x in open('tmp.txt', 'r').readlines()]
        os.system('rm tmp.txt')
    else:
        memory_gpu = False
        print("显存计算功能暂不支持windows操作系统")
    return memory_gpu


gpu_memory = get_gpu_memory()
if not gpu_memory:
    print("\ngpu free memory: {}".format(gpu_memory))
    gpu_list = np.argsort(gpu_memory)[::-1]

    gpu_list_str = ','.join(map(str, gpu_list))
    os.environ.setdefault("CUDA_VISIBLE_DEVICES", gpu_list_str)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

- 其中nvidia-smi -q -d Memory是查询所有 GPU 的内存信息，-q表示查询，-d是指定查询的内容。
- nvidia-smi -q -d Memory | grep -A4 GPU是截取 GPU 开始的 4 行
- nvidia-smi -q -d Memory | grep -A4 GPU | grep Free是提取Free所在的行，也就是提取剩余内存的信息
- nvidia-smi -q -d Memory | grep -A4 GPU | grep Free > tmp.txt是把剩余内存的信息保存到tmp.txt中
- 每隔一秒刷新显示 GPU 信息，可以使用watch -n 1 nvidia-smi 。

## 提高 GPU 的利用率
- nvidia-smi命令查看可以 GPU 的利用率
- Memory Usage表示显存的使用率
- Volatile GPU-Util表示计算 GPU 实际运算能力的利用率

- Memory Usage
    - 这个指标是由数据量主要是由模型大小，以及数据量的大小决定的。
    - 模型大小是由网络的参数和网络结构决定的，模型越大，训练反而越慢。
    - 我们主要调整的是每个 batch 训练的数据量的大小，也就是 batch_size。
    - 在模型结构固定的情况下，尽量将batch size设置得比较大，充分利用 GPU 的内存。

- Volatile GPU-Util
    - 上面设置比较大的 batch size可以提高 GPU 的内存使用率，却不一定能提高 GPU 运算单元的使用率。

- GPU 处理数据非常快，而 CPU 处理数据较慢。GPU 每接收到一个 batch 的数据，使用率就跳到逐渐升高，处理完这个 batch 的数据后，使用率又逐渐降低，等到 CPU 把下一个 batch 的数据传过来。

- 解决方法是：设置 Dataloader的两个参数：
    - num_workers：默认只使用一个 CPU 读取和处理数据。可以设置为 4、8、16 等参数。但线程数并不是越大越好。因为，多核处理需要把数据分发到每个 CPU，处理完成后需要从多个 CPU 收集数据，这个过程也是需要时间的。如果设置num_workers过大，分发和收集数据等操作占用了太多时间，反而会降低效率。
    - pin_memory：如果内存较大，建议设置为 True。
        - 设置为 True，表示把数据直接映射到 GPU 的相关内存块上，省掉了一点数据传输时间。
        - 设置为 False，表示从 CPU 传入到缓存 RAM 里面，再给传输到 GPU 上。